In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'data_fetch'))

In [2]:
import re
import pandas as pd
from yahoo_fin import stock_info as si
import pytz
from datetime import datetime, timedelta
from collections import Counter
from data_fetch.fetch_reddit_comments import get_existing_spreadsheet, authenticate_google_services
from data_fetch.credentials import TRADING_FOLDER_ID
from data_fetch.utils import bucket_by_time

import plotly.graph_objects as go

CREDS_PATH = os.path.join(os.path.dirname(os.getcwd()), 'Trading_Access.json')
TICKER_PATTERN = r'\b[A-Z]{1,5}\b'
ISRAEL_TZ = pytz.timezone("Asia/Jerusalem")
interval_minutes = 5

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [3]:
# Function to extract tickers along with timestamps
def extract_tickers(row):
    words = set(re.findall(TICKER_PATTERN, row.Body, re.IGNORECASE))  # Remove duplicates within each comment
    return [(word.upper(), row.Timestamp) for word in words]

def extract_tickers_uppercase(row):
    words = set(re.findall(TICKER_PATTERN, row.Body))  # Remove duplicates within each comment
    return [(word.upper(), row.Timestamp) for word in words]

def validate_tickers(ticker_list):
    """Validate extracted tickers using Yahoo Finance"""
    # all_tickers = set(si.tickers_dow() + si.tickers_sp500() + si.tickers_nasdaq() + si.tickers_other())
    all_tickers = set(si.tickers_dow() + si.tickers_sp500() + si.tickers_nasdaq())
    valid_tickers = set()
    invalid_tickers = set()

    for ticker in ticker_list:
        if ticker in all_tickers:
            valid_tickers.add(ticker)
        else:
            invalid_tickers.add(ticker)
    
    return valid_tickers, invalid_tickers

In [ ]:
date = '2025-02-18'
sheet_name = f'pennystocks_subreddit_{date}'
spreadsheet_id = get_existing_spreadsheet(sheet_name, TRADING_FOLDER_ID, CREDS_PATH)
gspread_client, _ = authenticate_google_services(CREDS_PATH)
spreadsheet = gspread_client.open_by_key(spreadsheet_id)
spreadsheet = spreadsheet.worksheet(date).get_all_values()

df = pd.DataFrame(spreadsheet[1:], columns=spreadsheet[0])
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df.sort_values(by='Timestamp', inplace=True)

comments = df[['Body', 'Timestamp']]

tickers_with_time = comments.apply(extract_tickers, axis=1)
tickers_with_time = [item for sublist in tickers_with_time for item in sublist]

ticker_list = list(dict.fromkeys([ticker for ticker, _ in tickers_with_time]))
valid_tickers, invalid_tickers = validate_tickers(ticker_list)

time_buckets = bucket_by_time([(ticker, ts.tz_localize(ISRAEL_TZ)) for ticker, ts in tickers_with_time], ISRAEL_TZ, interval_minutes)

# Keep only keys that are tuples of length 2 (date, bucket)
valid_keys = [key for key in time_buckets.keys() if isinstance(key, tuple) and len(key) == 2]

rows = []
    
# Count mentions in each time bucket
for (date, bucket) in sorted(valid_keys):
    
    start_time = (datetime.strptime(date, "%Y-%m-%d").replace(tzinfo=ISRAEL_TZ) + timedelta(minutes=bucket)).strftime("%H:%M")
    end_time = (datetime.strptime(date, "%Y-%m-%d").replace(tzinfo=ISRAEL_TZ) + timedelta(minutes=bucket + interval_minutes)).strftime("%H:%M")

    ticker_counts = Counter(time_buckets[(date, bucket)])
    filtered_counts = {ticker: count for ticker, count in ticker_counts.items() if ticker in valid_tickers}

    for ticker, count in filtered_counts.items():
        rows.append({"date": date, "start_time": start_time, "end_time": end_time, "ticker": ticker, "mentions": count})

# Convert to DataFrame
df_ticker_mentions = pd.DataFrame(rows, columns=["date", "start_time", "end_time", "ticker", "mentions"])

df_ticker_mentions["start_datetime"] = pd.to_datetime(df_ticker_mentions["date"] + " " + df_ticker_mentions["start_time"])
df_ticker_mentions["end_datetime"] = pd.to_datetime(df_ticker_mentions["date"] + " " + df_ticker_mentions["end_time"])
df_ticker_mentions.sort_values(by=['date', 'start_time', 'end_time', 'ticker'], inplace=True)
df_ticker_mentions = df_ticker_mentions[df_ticker_mentions.date == date].reset_index(drop=True)

df_ticker_mentions_daily = df_ticker_mentions.groupby('ticker')['mentions'].sum().reset_index()
df_ticker_mentions_daily.sort_values(by='mentions', ascending=False, inplace=True)

📄 Spreadsheet 'pennystocks_subreddit_2025-02-18' already exists: https://docs.google.com/spreadsheets/d/1mBUM8bPhV0wOpmujV9gTopSxfQcTWAXpU_Nky8mUYxg


In [15]:
upper_ticker = comments.apply(extract_tickers_uppercase, axis=1)
upper_tickers_with_time = [item for sublist in upper_ticker for item in sublist]
upper_ticker_list = list(dict.fromkeys([ticker for ticker, _ in upper_tickers_with_time]))
upper_valid_tickers, _ = validate_tickers(upper_ticker_list)

upper_df_ticker_mentions_daily = df_ticker_mentions_daily[df_ticker_mentions_daily.ticker.isin(upper_valid_tickers)]

In [17]:
upper_df_ticker_mentions_daily.head()

,ticker,mentions
220,SOBR,566
164,ON,520
5,ADTX,187
87,GO,172
224,SPGC,149


In [16]:
ticker = 'SOBR'
df_filtered = df_ticker_mentions[df_ticker_mentions.ticker == ticker][['end_datetime', 'mentions']]

# Create the Plotly figure
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_filtered['end_datetime'],
    y=df_filtered['mentions'],
    mode='lines+markers',  # Line chart with markers at each point
    name=ticker,
    line=dict(width=2),
    marker=dict(size=6)
))

# Customize layout
fig.update_layout(
    title=f"Mentions of {ticker} Over Time",
    xaxis_title="Time",
    yaxis_title="Mentions",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template="plotly_white"
)

# Show plot
fig.show()


In [40]:
upper_df_ticker_mentions_daily.iloc[:50]

,ticker,mentions
221,SOBR,560
165,ON,508
6,ADTX,177
88,GO,169
225,SPGC,150
145,MGOL,143
52,DGLY,138
44,CNSP,131
19,BACK,130
227,STAI,121
